# VA Exception Generator

In [ ]:
import sys
import json
import requests, urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

from insightvm_authen import InsightvmAuthentication

In [ ]:
HOSTNAME = InsightvmAuthentication().getHostname()
HEADER = InsightvmAuthentication().getRequestHeader()
# HOSTNAME, HEADER

## 1. Get Vulnerabilities Detail from ID

In [ ]:
def getVulnerabilityDetail(vulnerabilitiesTextID=""):
    try:
        getVulnerabilityDetailURL = HOSTNAME + "api/3/vulnerabilities/" + str(vulnerabilitiesTextID)
        print("\nGetting Vulnerability Detail")
        print("Hostname : " + getVulnerabilityDetailURL)

        r = requests.get(getVulnerabilityDetailURL, headers=HEADER, verify=False)

        if r.status_code == 200:
            r = r.json()
            return {r["id"]: r["title"]}
        else:
            print("[%d] Request Connection Error\n" % r.status_code)
            sys.exit()
    except:
        print("[Error] getVulnerabilityDetail Function\n")
        sys.exit()

In [ ]:
vulnerabilitiesTextID = "ssl-only-weak-ciphers"  # [1, 2, 3, 5]

In [ ]:
vulnerabilityDetail = getVulnerabilityDetail(vulnerabilitiesTextID=vulnerabilitiesTextID)
vulnerabilityDetail

## 2. VA Exception

### scopeTypeID

    0 : Global
    1 : Site
    2 : Asset
    3 : Asset Group
    4 : Instance

### exceptionReasonID

    0 : False Positive
    1 : Compensating Control
    2 : Acceptable Use
    3 : Acceptable Risk"
    4 : "Other"

In [ ]:
def runVulnerabilityException(vulnerabilitiesTextID=None, scopeTypeID=0, scopeID=None, instanceKey="", submitVAException={}):
    runVulnerabilityExceptionURL = HOSTNAME + "api/3/vulnerability_exceptions"
    print("\nRunning Except Vulnerability")
    print("Hostname : " + runVulnerabilityExceptionURL)

    scopeType = ["Global", "Site", "Asset", "Asset Group", "Instance"]
    exceptionReason = ["False Positive", "Compensating Control", "Acceptable Use", "Acceptable Risk", "Other"]

    payloads = {
        "expires": "",
        "scope": {
            "id": str(scopeID),
            "vulnerability": str(vulnerabilitiesTextID),
            "key": str(instanceKey),
            "type": scopeType[scopeTypeID]
        },
        "state": "Approved",
        "submit": {
            "comment": submitVAException["description"],
            "reason": exceptionReason[submitVAException["exception_reason"]]
        }
    }

    r = requests.post(runVulnerabilityExceptionURL, data=json.dumps(payloads), headers=HEADER, verify=False)

    if r.status_code == 201:
        return r.json()["id"]
    else:
        print("[%d] Request Connection Error\n" % r.status_code)
        sys.exit()

In [ ]:
vulnerabilitiesTextID = "jre-vuln-cve-2016-3427"
instanceKey = "/home/ctmagent/ctm/cm/AFT/JRE/lib/rt.jar"

In [ ]:
exceptionReasonID = 0
exceptionDescription = "Test"
submitVAException = {
    "description": exceptionReasonID,
    "exception_reason": exceptionDescription
}

In [ ]:
scopeTypeID = 2

### A. Input multiple AssetIDs

In [ ]:
assetIDs = [1, 2, 3, 4, 5]
assetIDs

Or

### B. Get all assets by Vulnerability Text ID

In [ ]:
def getVulnerabilityAffectedAsset(vulnerabilitiesTextID=""):
    try:
        getVulnerabilityAffectedAssetURL = HOSTNAME + "api/3/vulnerabilities/" + str(vulnerabilitiesTextID) + "/assets"
        print("\nGetting Vulnerability Affected Assets")
        print("Hostname : " + getVulnerabilityAffectedAssetURL)

        r = requests.get(getVulnerabilityAffectedAssetURL, headers=HEADER, verify=False)

        if r.status_code == 200:
            return r.json()["resources"]
        else:
            print("[%d] Request Connection Error\n" % r.status_code)
            sys.exit()
    except:
        print("[Error] getVulnerabilityAffectedAsset Function\n")
        sys.exit()

In [ ]:
assetIDs = getVulnerabilityAffectedAsset(vulnerabilitiesTextID=vulnerabilitiesTextID)
assetIDs

### Run Exception with Multiple AssetIDs

In [ ]:
for assetID in assetIDs:
    runVAExceptionWithInstanceKeyByAssetID(vulnerabilitiesTextID=vulnerabilitiesTextID, scopeTypeID=scopeTypeID, instanceKey=instanceKey, assetID=str(assetID), submitVAException=submitVAException)

### Run Exception with Vulnerability Instance Key

In [ ]:
def checkHaveVulnInstanceInAsset(assetID="", vulnerabilitiesTextID="", instanceKey=""):
    print("\nChecking Instance key in Your asset : %s" % str(assetID))
    
    getVulnerabilityInstanceOfAssetURL = HOSTNAME + "api/3/assets/" + str(assetID) + "/vulnerabilities/" + str(vulnerabilitiesTextID)
    print("\nGetting Vulnerability Instances of Asset : %s" % str(assetID))
    print("Hostname : " + getVulnerabilityInstanceOfAssetURL)

    r = requests.get(getVulnerabilityInstanceOfAssetURL, headers=HEADER, verify=False)

    if r.status_code == 200:
        r = r.json()["results"]
        for insta in r:
            if "key" in insta and insta["key"] == instanceKey:
                return True
        return False
    else:
        print(r.status_code)    

In [ ]:
vulnerabilitiesTextID = "jre-vuln-cve-2016-3427"
instanceKey = "/home/ctmagent/ctm/cm/AFT/JRE/lib/rt.jar"

In [ ]:
exceptionReasonID = 0
exceptionDescription = "Test"
submitVAException = {
    "description": exceptionReasonID,
    "exception_reason": exceptionDescription
}

In [ ]:
scopeTypeID = 4
scopeID= "999"

In [ ]:
assetID = "1234"

In [ ]:
if checkHaveInstanceInAsset(assetID=str(assetID), vulnerabilitiesTextID=vulnerabilitiesTextID, instanceKey=instanceKey):
    print("[Found] Instance key in Your asset.")
    vaExceptionID = runVulnerabilityException(vulnerabilitiesTextID=vulnerabilitiesTextID, scopeTypeID=scopeTypeID, scopeID=assetID, instanceKey=instanceKey, submitVAException=submitVAException)
    print("[Successful] Vulnerabilities Exceptions ID : " + str(vaExceptionID))
else:
    print("[Not Found] Instance key in Your asset.")


# Get Vulnerability Exceptions Detail By ID

In [ ]:
vaExceptionID = ""

In [ ]:
getVulnerabilityExceptionsDetailURL = HOSTNAME + "api/3/vulnerability_exceptions/" + str(vaExceptionID)
print("Hostname : " + getVulnerabilityExceptionsDetailURL)

r = requests.get(getVulnerabilityExceptionsDetailURL, headers=HEADER, verify=False)
r = r.json()

In [ ]:
r

In [ ]:
filepath = "outputs/getVulnerabilityExceptionsDetail.json"

In [ ]:
with open(filepath, 'w', encoding="utf8") as f:
    f.write(json.dumps(r, indent=4))
print("\n[Success Exporting] " + filepath)

# Delete Vulnerability Exception By ID

In [ ]:
vaExceptionID = ""

In [ ]:
deleteVulnerabilityExceptionsURL = HOSTNAME + "api/3/vulnerability_exceptions/" + str(vaExceptionID)
print("Hostname : " + deleteVulnerabilityExceptionsURL)

r = requests.delete(deleteVulnerabilityExceptionsURL, headers=HEADER, verify=False)

In [ ]:
r.status_code == 200